In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt
import re


In [ ]:
dbname = mongo_utils.get_database()
collection_name = dbname["dnesbg_scrape_1"]

In [ ]:
url='https://www.dnes.bg/news.php?today'


In [ ]:
feeds = feedparser.parse(url)

In [ ]:
#print(feeds)
manin_page = requests.get(url)
man_page_data = manin_page.text
main_page = BeautifulSoup(man_page_data, 'html.parser')
pln_txt=main_page.find("p", class_="pages")#.find_all('p')
pln_txt=pln_txt.find_all("a", href=True)
pgsnmb=[]
for pgs in pln_txt:
    nmbr=pgs['href'].rsplit('=',1)[-1]
    pgsnmb.append(nmbr)
pgsnmb.sort()
last_page=pgsnmb[-1]


In [ ]:
for page in range(1,int(last_page)+1):
    try:
        page=(url + "&page=" + str(page))
        manin_page = requests.get(page)
        man_page_data = manin_page.text
        main_page = BeautifulSoup(man_page_data, 'html.parser')
        pln_txt=main_page.find("div", id="c")
        pln_txt=pln_txt.find("div", id="c1")
        for n in pln_txt.find_all("div", class_="b2"):
           try:
            st_url=n.find("a", href=True)
            nlink=("https://www.dnes.bg" + st_url['href'])
            print(nlink)
            dt_obj=n.find("div", class_="date").get_text()
            date_posted=dt.strptime(dt_obj, '%d.%m.%Y %H:%M').timestamp()
            art_page = requests.get(nlink)
            art_page_data = art_page.text
            art_page_data=BeautifulSoup(art_page_data, 'html.parser')
            news_title=art_page_data.find("h1", class_="title").get_text()
            art_page_data=art_page_data.find("div", id="art_start")
            if art_page_data.find("div", class_="article_related article_related_right"):#.decompose()
               art_page_data.find("div", class_="article_related article_related_right").decompose()
            text=art_page_data.get_text().replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', ' ').replace('  ', ' ')
            news_payload = {
               "date_posted" : date_posted,
               "news_link" : nlink,
               "news_text" : text,
               "news_title" : news_title
            }           
            collection_name.insert_many([news_payload])
            print(news_payload)
           except:
             pass
    except:
        pass